# ДЗ 5 в процессе
Приложен ноутбук, в котором реализованы функции для генерации из большого
датасета в меньшую его копию. Вам нужно перенести функции из этого
ноутбука в класс датасет и сделать следующее:
a. Сгенерировать меньший датасет из 8-10 классов движения
b. Обучить уже существующую модель (предварительно проанализировав
какие параметры модели нужно изменить)
c. Изменить модель: посмотреть зависимость от количества LSTM модулей
в нашей модели
d. Сгенерировать другой датасет с меньшим количеством “кадров” в серии
и сравнить улучшилось или ухудшилось качество предсказания.
Провести несколько таких итераций, дать свою оценку уменьшению и
увеличению кадров, назвать оптимальное, на ваш взгляд, их количество.
Желательно сделать так, чтобы длина последовательности
передавалась как атрибут класса.

In [1]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader, Dataset
from torch import optim
import time
import math
import os

use_cuda = torch.cuda.is_available()
# device = torch.device("cpu")
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cpu


/home/evm/anaconda3/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [9]:
data_path = "./data/nturgb+d_skeletons/"
broken_files_path = "./data/NTU_RGBD_samples_with_missing_skeletons.txt"

In [3]:
# training_subjects = list(range(0, 28)) #количество людей выполняющих действия
training_classes = sorted([2, 8, 10, 22, 27, 21, 43, 52, 55]) #классы которые будем использовать для обучения, полный список прдставлен тут https://github.com/shahroudy/NTURGB-D
LABELS = {x: training_classes[x] for x in range(len(training_classes))}
training_cameras = [1, 2, 3] 

# max_body_true = 1
# max_body_kinect = 1

num_joint = 25
max_frame = 300

In [11]:
class SkeletonData:
    def __init__(self, dpath):
        self.data_path = dpath + '/nturgb+d_skeletons/'
        self.broken_files_path = dpath + '/NTU_RGBD_samples_with_missing_skeletons.txt'
        self.training_subjects = list(range(0, 28))
        self.training_cameras = [1, 2, 3]
        self.training_classes = [8, 10, 21, 22, 23, 25, 26, 27, 43, 52]
        self.working_files_with_labels, self.action_classes = self.read_data(self.data_path, self.broken_files_path)
    

    def read_data(self, data_path, broken_files_path):
        labels = []
        files = []
        action_classes = {}
        counter = 0
        files_counter = {}

        with open(broken_files_path, 'r') as f:
            broken_files = f.read().split("\n")

        raw_files = os.listdir(data_path)
        num_frames = 0

        for filename in raw_files:
            if filename not in broken_files:
                action_class = int(filename[filename.find('A') + 1:filename.find('A') + 4])
                subject_id = int(filename[filename.find('P') + 1:filename.find('P') + 4])
                camera_id = int(filename[filename.find('C') + 1:filename.find('C') + 4])
                if action_class in self.training_classes and camera_id in self.training_cameras:  #and subject_id in training_subjects:
                    if action_class in action_classes:
                        if files_counter[action_class] < 120:
                            files.append([filename,action_classes[action_class]])
                            files_counter[action_class] = files_counter[action_class] + 1
                    else:
                        action_classes.update({action_class : counter})
                        files_counter.update({action_class : 1})
                        counter+=1
                        files.append([filename,action_classes[action_class]])
    #                     labels.append([action_class])
#         print("action classes: ", action_classes)
#         print("action files: ", files_counter)

        return files, action_classes
    
    
    @staticmethod
    def read_skeleton_filter(file):
        with open(file, 'r') as f:
            skeleton_sequence = {}
            skeleton_sequence['numFrame'] = int(f.readline())
            skeleton_sequence['frameInfo'] = []
            for t in range(skeleton_sequence['numFrame']):
                frame_info = {}
                frame_info['numBody'] = int(f.readline())
                frame_info['bodyInfo'] = []

                for m in range(frame_info['numBody']):
                    body_info = {}
                    body_info_key = [
                        'bodyID', 'clipedEdges', 'handLeftConfidence',
                        'handLeftState', 'handRightConfidence', 'handRightState',
                        'isResticted', 'leanX', 'leanY', 'trackingState'
                    ]
                    body_info = {
                        k: float(v)
                        for k, v in zip(body_info_key, f.readline().split())
                    }
                    body_info['numJoint'] = int(f.readline())
                    body_info['jointInfo'] = []
                    for v in range(body_info['numJoint']):
                        joint_info_key = [
                            'x', 'y', 'z', 'depthX', 'depthY', 'colorX', 'colorY',
                            'orientationW', 'orientationX', 'orientationY',
                            'orientationZ', 'trackingState'
                        ]
                        joint_info = {
                            k: float(v)
                            for k, v in zip(joint_info_key, f.readline().split())
                        }
                        body_info['jointInfo'].append(joint_info)
                    frame_info['bodyInfo'].append(body_info)
                skeleton_sequence['frameInfo'].append(frame_info)

        return skeleton_sequence
    
    

    def read_xyz(self, file, max_body=1, num_joint=25):
        seq_info = self.read_skeleton_filter(file)
        data = np.zeros((max_body, seq_info['numFrame'], num_joint, 3))
        for n, f in enumerate(seq_info['frameInfo']):
            for m, b in enumerate(f['bodyInfo']):
                for j, v in enumerate(b['jointInfo']):
                    if m < max_body and j < num_joint:
                        data[m, n, j, :] = [v['x'], v['y'], v['z']]

                    else:
                        pass

        return data
    
    

    def create_coords_blocks(self, test_file, chonk_len):   
        frame_counter = 0
        new_labels = []
        new_frames = []
        blocks = []

        test_frames = self.read_xyz(str(self.data_path) + test_file[0])[0]
        label = test_file[1]
        slice_len = chonk_len * int(len(test_frames)/chonk_len)


        for index in range(len(test_frames[:slice_len])):
            frame_counter += 1
            new_frames.append(test_frames[index].flatten())
            if frame_counter == chonk_len:
                frame_counter = 0
                blocks.append(np.array(new_frames))
                new_labels = new_labels + [label]
                new_frames = []


        return blocks, new_labels
    

    def create_dataset(self, chonk_len):
        data = []
        labels = []
        ##########################################################################
        numbers = {x: 0 for x in range(len(self.action_classes))} #####
        ##################################################################
        for file in self.working_files_with_labels:
            frames_blocks, label = self.create_coords_blocks(file, chonk_len)
            if label != [] and numbers[label[0]] <= 150:
                numbers[label[0]] = numbers[label[0]] + len(label)
                data = data + frames_blocks
                labels = labels + label
        data_np = np.asarray(data)
        labels_np = np.asarray(labels)

        data_sq = data_np.reshape(len(data_np), -1)
        test_data = pd.DataFrame(data_sq)
        test_labels = pd.DataFrame(labels_np)
        test_data['labels'] = test_labels
        
        return test_data

In [10]:
dataset = Skeleton_Dset(data_path=data_path, broken_files_path=broken_files_path, 
                           training_classes=training_classes,num_joint = 25, 
                           max_frame = 300, transform=None)

action classes:  {21: 0, 27: 1, 10: 2, 22: 3, 43: 4, 52: 5, 55: 6, 8: 7, 2: 8}
action files:  {21: 56, 27: 57, 10: 57, 22: 50, 43: 41, 52: 58, 55: 58, 8: 59, 2: 57}


In [14]:
skeletons50 = SkeletonData('./data').create_dataset(50)
skeletons50.head()

,0,1,2,3,4,5,6,7,8,9,...,3741,3742,3743,3744,3745,3746,3747,3748,3749,labels
0,-0.258776,-0.218672,3.457855,-0.258671,0.105829,3.516608,-0.257890,0.423046,3.561671,-0.251553,...,-0.390676,0.220879,2.772770,-0.135334,-0.309912,3.262463,-0.086270,-0.253832,3.318211,0
1,-0.321709,-0.902678,4.208284,-0.325072,-0.627424,4.223121,-0.326405,-0.354732,4.226923,-0.328998,...,-0.539932,-1.067394,4.082897,-0.183971,-1.141637,4.021587,-0.170118,-1.092657,4.083500,1
2,-0.376011,-0.482615,3.528463,-0.389711,-0.297486,3.503011,-0.400182,-0.112305,3.468939,-0.410824,...,-0.859159,-0.129772,3.417000,-0.734264,-0.237363,3.354874,-0.673244,-0.238646,3.346857,0
3,0.196974,-0.502818,4.364193,0.179242,-0.250589,4.350000,0.160283,0.000277,4.330863,0.166488,...,0.258430,-0.145001,4.226722,0.444555,0.074409,4.099661,0.380363,0.054859,4.116461,2
4,0.021622,-0.730864,3.395480,0.030033,-0.435900,3.345957,0.039064,-0.148019,3.290263,0.070492,...,0.119964,-0.077448,2.922167,0.011526,0.086074,2.985360,-0.024904,0.102671,3.003286,3


In [15]:
skeletons50.to_csv("./skels50.csv", index = False)
